### Import libraries

In [23]:
%matplotlib inline

import pandas as pd
import numpy as np

from scipy.stats import trim_mean
import wquantiles
from statsmodels import robust

#### Load data

In [4]:
AIRLINE_STATS_CSV   = '/data/airline_stats.csv'
KC_TAX_CSV          = '/data/kc_tax.csv.gz'
LC_LOANS_CSV        = '/data/lc_loans.csv'
AIRPORT_DELAYS_CSV  = '/data/dfw_airline.csv'
SP500_DATA_CSV      = '/data/sp500_data.csv.gz'
SP500_SECTORS_CSV   = '/data/sp500_sectors.csv'
STATE_CSV           = 'data/state.csv'

##### Estimates of Location

Example: Location estimates of Population and Murder Rates

In [5]:
state = pd.read_csv(STATE_CSV)
state.head()

,State,Population,Murder.Rate,Abbreviation
0,Alabama,4779736,5.7,AL
1,Alaska,710231,5.6,AK
2,Arizona,6392017,4.7,AZ
3,Arkansas,2915918,5.6,AR
4,California,37253956,4.4,CA


`Mean`, `Median` and `Trimmed Mean`

In [13]:
# Mean Calculation using pandas function
mean = state['Population'].mean()
print(f'Mean of Population = {mean}')

# Median Calculation using pandas function
median = state['Population'].median()
print(f'Median of Population = {median}')

# Trimmed Mean Calculation using scipy.stats
trimmed_mean = trim_mean(state['Population'], 0.1)
print(f'Trimmed Mean of Population = {trimmed_mean}')

Mean of Population = 6162876.3
Median of Population = 4436369.5
Trimmed Mean of Population = 4783697.125


<p>Proof that how outlier is reduced from mean to median</p> <br>
Mean > Trimmed Mean > Median


`Weighted mean` and `Weighted Median`

In [16]:
# Weighted mean calculation using Numpy
weighted_mean = np.average(
    state['Murder.Rate'],
    weights = state['Population']
)
print(f'Weighted Mean of Murder rate where weights are Population = {weighted_mean}')

# Weighted Median calculation using wquantiles library
weighted_median = wquantiles.median(
    state['Murder.Rate'],
    weights = state['Population']
)
print(f'Weighted Median of Murder rate where weights are Population = {weighted_median}')


Weighted Mean of Murder rate where weights are Population = 4.445833981123393
Weighted Median of Murder rate where weights are Population = 4.4


### Estimates of Variability

In [6]:
state.head(8)

,State,Population,Murder.Rate,Abbreviation
0,Alabama,4779736,5.7,AL
1,Alaska,710231,5.6,AK
2,Arizona,6392017,4.7,AZ
3,Arkansas,2915918,5.6,AR
4,California,37253956,4.4,CA
5,Colorado,5029196,2.8,CO
6,Connecticut,3574097,2.4,CT
7,Delaware,897934,5.8,DE


`Standard Deviation`, `Interquartile range` and `Median absolute Deviation`

In [25]:
print(f"Standard Deviation of Population: {state['Population'].std()}")
print(f"Interquartile range { state['Population'].quantile(0.75) -  state['Population'].quantile(0.25) }")
# Method 1:
print(f"Mean Absolute Deviation {robust.scale.mad(state['Population'])}")
# Method 2:

'''
The value 0.6744897501960817 is the constant scaling factor for the median absolute deviation (MAD) to estimate the 
standard deviation of a normal distribution. This value is approximately equal to the inverse of the cumulative 
distribution function (CDF) at 0.75 for a standard normal distribution, which is commonly used in statistics.

In simpler terms, this constant is used to scale the median absolute deviation so that it can be interpreted in a way 
similar to the standard deviation of a normal distribution. It helps in comparing the variability of different datasets, 
especially when dealing with distributions that are not necessarily normally distributed.
'''

pd_mad = abs(
    state['Population'] - state['Population'].median()
).median() / 0.6744897501960817

print(f"Mean Absolute Deviation by Pandas: {pd_mad}")

Standard Deviation of Population: 6848235.347401142
Interquartile range 4847308.0
Mean Absolute Deviation 3849876.1459979336
Mean Absolute Deviation by Pandas: 3849876.1459979336
